<a href="https://colab.research.google.com/github/DerizhyLouise/RevoU-Data-Analytics-Capstone-Project/blob/main/DataAnalytics-Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/RevoU - Capstone Project/Data Cleaning - Python'

/content/drive/MyDrive/RevoU - Capstone Project/Data Cleaning - Python


In [ ]:
orderDf = pd.read_csv('orders.csv')
custDf = pd.read_csv('customers.csv')
salesDf = pd.read_csv('sales.csv')
prodDf = pd.read_csv('products.csv')

orderCustDf = pd.merge(orderDf, custDf, on='customer_id', how='inner')
orderSalesDf = pd.merge(orderDf, salesDf, on='order_id', how='inner')
orderSalesProdDf = pd.merge(orderSalesDf, prodDf, on='product_id', how='inner')

In [ ]:
orderDf["order_date"] = pd.to_datetime(orderDf["order_date"])
orderDf["delivery_date"] = pd.to_datetime(orderDf["delivery_date"])
orderCustDf["order_date"] = pd.to_datetime(orderCustDf["order_date"])
orderCustDf["delivery_date"] = pd.to_datetime(orderCustDf["delivery_date"])
orderCustDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1000 non-null   int64         
 1   customer_id    1000 non-null   int64         
 2   payment        1000 non-null   int64         
 3   order_date     1000 non-null   datetime64[ns]
 4   delivery_date  1000 non-null   datetime64[ns]
 5   customer_name  1000 non-null   object        
 6   gender         1000 non-null   object        
 7   age            1000 non-null   int64         
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 70.3+ KB


In [ ]:
orderCustDf.head()

,order_id,customer_id,payment,order_date,delivery_date,customer_name,gender,age
0,1,64,30811,2021-08-30,2021-09-24,Annabella Devote,Intersex,75
1,2,473,50490,2021-02-03,2021-02-13,Lori Briars,Male,61
2,221,473,49366,2021-08-03,2021-08-17,Lori Briars,Male,61
3,3,774,46763,2021-10-08,2021-11-03,Ellynn Korba,Intersex,34
4,353,774,46198,2021-03-30,2021-04-05,Ellynn Korba,Intersex,34


## EDA (Exploratory Data Analysis)

# Descriptive Analytics

In [ ]:
orderCustDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       1000 non-null   int64         
 1   customer_id    1000 non-null   int64         
 2   payment        1000 non-null   int64         
 3   order_date     1000 non-null   datetime64[ns]
 4   delivery_date  1000 non-null   datetime64[ns]
 5   customer_name  1000 non-null   object        
 6   gender         1000 non-null   object        
 7   age            1000 non-null   int64         
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 70.3+ KB


How many order per customer?

In [ ]:
order_count_per_customer = orderCustDf.groupby('customer_id')['order_id'].count().reset_index()
order_count_per_customer = order_count_per_customer.rename(columns={'order_id': 'order_count'})
order_count_per_customer = order_count_per_customer.sort_values(by="order_count", ascending=False)
print(order_count_per_customer)

     customer_id  order_count
351          571            6
212          348            5
347          566            5
166          282            5
410          664            5
..           ...          ...
224          368            1
222          366            1
221          365            1
220          364            1
616         1000            1

[617 rows x 2 columns]


How many order per month?

In [ ]:
orderCustDf['order_month'] = orderCustDf['order_date'].dt.to_period('M')
order_count_per_customer_month = orderCustDf.groupby('order_month').size().reset_index(name='order_count')
print(order_count_per_customer_month)

  order_month  order_count
0     2021-01          109
1     2021-02           94
2     2021-03          118
3     2021-04          101
4     2021-05           84
5     2021-06          103
6     2021-07          105
7     2021-08          107
8     2021-09           98
9     2021-10           81


How many order per product?

In [ ]:
order_count_per_product = orderSalesProdDf.groupby(['product_id', 'product_name', 'product_type'])['order_id'].count().reset_index()
order_count_per_product = order_count_per_product.rename(columns={'order_id': 'order_count'})
order_count_per_product = order_count_per_product.sort_values(by="order_count", ascending=False)
print(order_count_per_product)

      product_id   product_name product_type  order_count
76            78        Flannel        Shirt           13
725          740        Peacoat       Jacket           12
824          843         Chinos     Trousers           11
570          579         Bomber       Jacket           11
1014        1038    Relaxed Leg     Trousers           10
...          ...            ...          ...          ...
454          461         Puffer       Jacket            1
186          188  Camp Collared        Shirt            1
746          763        Peacoat       Jacket            1
432          438          Denim       Jacket            1
592          602       Cardigan       Jacket            1

[1233 rows x 4 columns]


How many order per customer based on month?

In [ ]:
orderCustDf['order_month'] = orderCustDf['order_date'].dt.to_period('M')
order_count_per_customer_month = pd.pivot_table(orderCustDf, values='order_id', index='customer_id', columns='order_month', aggfunc='count', fill_value=0)
order_count_per_customer_month.columns = order_count_per_customer_month.columns.strftime('%Y-%m')
print(order_count_per_customer_month)

order_month  2021-01  2021-02  2021-03  2021-04  2021-05  2021-06  2021-07  \
customer_id                                                                  
1                  0        1        1        1        0        0        0   
7                  0        0        0        0        1        0        0   
10                 0        0        1        0        0        0        0   
11                 0        0        0        0        1        0        0   
12                 0        0        0        0        0        1        0   
...              ...      ...      ...      ...      ...      ...      ...   
994                1        0        0        0        0        0        0   
995                0        0        0        0        0        0        0   
996                0        0        1        0        0        0        0   
998                1        1        1        0        0        0        1   
1000               0        0        0        0        0        

#Business Question

What is the percentage of customers who have become inactive over the last three months?
Assuming today is 1 November 2021

In [ ]:
currentDate = pd.to_datetime('2021-11-01')
last3Months = currentDate - pd.DateOffset(months=3)
lastOrder = orderDf[orderDf['order_date'] >= last3Months]
lastOrder = lastOrder['customer_id'].unique()
print(lastOrder)

[  64  774  800  659  204  468  513  469  206  358   51  849  454  493
  631   68  181  874  448  803  908  291  607  160  321  936  629  539
  298  528  912  818  357  727  674  208  867  443  574  315  715  240
  833  415  630  640   15  568  644   28   62  678  526  555  473  368
  664  406  107  497  996  133  767  530  909  637  179  571  605  313
  742  878   34  828  303  882  799  201  364  426  499  207   58  758
  747  679   95  933   41  686  367  485  341  348  673  732   52  870
   96  556  623  146  508   13  331   94  980   47  427  589  129  729
  918  745  691  776   20  419  739  269  286  553  185  163  566  123
  371  757  810  452  227  111  638   77  816  352  690  937  737  648
  125  282  223  244  986   79  343  523   49  490   26  330  922  632
  583   54  783  402  308  921  549  735  808  534  904  327  886  548
  794  417  328  787  924  456  889  463  956  851  884  941  230  687
  474  225  387  755  401  135  447  754  211  750  693  535  198   83
  955 

In [ ]:
lastCust = custDf[custDf['customer_id'].isin(lastOrder)]
print(lastCust)

     customer_id      customer_name    gender  age
12            13    Marabel Swinfon      Male   42
14            15     Avril Rossiter  Intersex   34
15            16  Gabbie Aldwinckle      Male   75
19            20       Gigi Kalaher  Intersex   55
25            26   Timothy Compston  Intersex   50
..           ...                ...       ...  ...
983          984     Devin Lindmark    Female   57
985          986  Theodosia Postins  Intersex   38
994          995      Leandra Derks  Intersex   63
995          996     Elvira Sarfati  Intersex   59
999         1000   Mandel Fairbanks      Male   71

[251 rows x 4 columns]


In [ ]:
activeCustCount = lastCust.shape[0]
print("Total active users in last three month is", activeCustCount, "users")

Total active users in last three month is 251 users


In [ ]:
totalCust = custDf.shape[0]
activePercentage = activeCustCount/totalCust * 100
unactiveCustCount = abs(totalCust - activeCustCount)
unactivePercentage = unactiveCustCount/totalCust * 100

print("Total customers who have became inactive over the last three months is", unactiveCustCount, "customers")
print("This count is about ", unactivePercentage, "% of all customers count", sep="")

Total customers who have become inactive over the last three months is 749 customers
This count is about 74.9% of all customers count
